In [1]:
import requests
import json
# Aho-Corasick
from ahocorapy.keywordtree import KeywordTree

In [2]:
#import for normalization
# importing NLTK libarary stopwords 
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
# for removing number
import re
# for remove Punctuation
import string

# import openpyxl module for excel
import openpyxl 

[nltk_data] Downloading package stopwords to /home/maya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/maya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# functions  data sets defenation
#### first rule set 
conditions_A_1={"keywords":"korona, Dünya Sağlık Örgütü",
"tags":"news","categories":"Sağlık, Yaşam,Dünya"}
rule_A_1={"name":"Korona Virüs Dünya Sağlık Örgütü Duyuru","conditions":conditions_A_1}
####
conditions_A_2={"keywords":"Dünya Sağlık Örgütü, Soumya Swaminathan",
"tags":"news","name":"F5 HABER"}
rule_A_2={"name":"Dünya Sağlık Örgütü ve Soumya Swaminathan","conditions":conditions_A_2}
#### 
ruleSet_A={"name":"Dünya Sağlık Örgütü Duyuruları","rules":[rule_A_1,rule_A_2]}

#### second rule set
conditions_B_1={"keywords":"coronavirus, Covid-19",
"type":"Foreign","lang":"en"}
rule_B_1={"name":"coronavirus news","conditions":conditions_B_1}

conditions_B_2={"keywords":"ilaç, firma,korona",
"lang":"tr"}
rule_B_2={"name":"korona ilaç","conditions":conditions_B_2}


ruleSet_B={"name":"coronavirus english news and drug","rules":[rule_B_1,rule_B_2]}

####
ruleSets=[ruleSet_A,ruleSet_B]
ruleSets



[{'name': 'Dünya Sağlık Örgütü Duyuruları',
  'rules': [{'name': 'Korona Virüs Dünya Sağlık Örgütü Duyuru',
    'conditions': {'keywords': 'korona, Dünya Sağlık Örgütü',
     'tags': 'news',
     'categories': 'Sağlık, Yaşam,Dünya'}},
   {'name': 'Dünya Sağlık Örgütü ve Soumya Swaminathan',
    'conditions': {'keywords': 'Dünya Sağlık Örgütü, Soumya Swaminathan',
     'tags': 'news',
     'name': 'F5 HABER'}}]},
 {'name': 'coronavirus english news and drug',
  'rules': [{'name': 'coronavirus news',
    'conditions': {'keywords': 'coronavirus, Covid-19',
     'type': 'Foreign',
     'lang': 'en'}},
   {'name': 'korona ilaç',
    'conditions': {'keywords': 'ilaç, firma,korona', 'lang': 'tr'}}]}]

In [4]:
#mormalization functions
def getLanguageName(lang):
    if lang=="en":
        return "english"
    elif lang=="tr":
        return "turkish"
def removeStopwords(txt,lang):
    text_tokens = word_tokenize(txt)
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words(getLanguageName(lang))]
    return ' '.join(tokens_without_sw)  
def convertToLowerCase(txt):
    return txt.lower()
def removePunctuation(txt):
    return txt.translate(str.maketrans('', '', string.punctuation))
def normalizeText(txt,lang):
    normalizedText=removePunctuation(txt)
    normalizedText=convertToLowerCase(normalizedText)
    normalizedText=removeStopwords(normalizedText,lang)
    return normalizedText

In [5]:
# Aho-Corasick functions
def getAhoCorasickTree(keywords):
    kwtree = KeywordTree(case_insensitive=True)
    for keyword in keywords:
        kwtree.add(keyword)
    kwtree.finalize()
    return kwtree
def getMatchedWords(txt,ahoCorasickTree):
    resultSet=set()
    results = kwtree.search_all(txt)
    for result in results:
        resultSet.add(result[0])
    return list(resultSet)
    
def getFilteredNews(allNews,ahoCorasickTree):
    #return  news with matched words prop that contains at least one matched word 
    filteredNews=[]
    for news in allNews:
        matchedWords=getMatchedWords(news["text"],ahoCorasickTree)
        if len(matchedWords)!=0:
            news["matchedWords"]=matchedWords
            filteredNews.append(news)
    return filteredNews
            
            

In [6]:
# functions realted to news
def getText(news):
    if "description" in news:
        return news["title"]+news["description"]+news["content"]
    return news["title"]+news["content"]
def getNewsWithText(allNews):
    for news in allNews:
        news["text"]=normalizeText(getText(news),news["lang"])
    return allNews
def matchAllRuleSetKeywords(ruleSetKeywords,matchedWords):
    return all(word in matchedWords for word in ruleSetKeywords)
def matchAllConditions(news,rule):
    ruleKeywords=getRuleKeywords(rule)
    if len(ruleKeywords)> len(news["matchedWords"]):
        return False    
    if "keywords" in rule["conditions"]:
        if not matchAllRuleSetKeywords(ruleKeywords,news["matchedWords"]):
            return False
    if "lang" in rule["conditions"]:
        if "".join(news["lang"])  not in rule["conditions"]["lang"]:
            return False
    if "tags" in rule["conditions"]:
        if "".join(news["tags"])  not in rule["conditions"]["tags"]:
            return False
    if "name" in rule["conditions"]:
        if "".join(news["name"])  not in rule["conditions"]["name"]:
            return False
    if "type" in rule["conditions"]:
        if "".join(news["type"])  not in rule["conditions"]["type"]:
            return False
    if "categories" in rule["conditions"]:
        if "".join(news["categories"])  not in rule["conditions"]["categories"]:
            return False
    return True
def setRuleMatchedNews(allNews,rule,ruleSetName):
    for news in allNews:
        if(matchAllConditions(news,rule)):
            # store matched rules name as value to key with dataset name
            if ruleSetName in news:
                news[ruleSetName]= news[ruleSetName]+","+rule["name"]
            else:
                news[ruleSetName]=rule["name"]

def setRuleSetMatchedNews(allNews,ruleSet):
    for rule in ruleSet["rules"]:
        setRuleMatchedNews(allNews,rule,ruleSet["name"])
def  getRuleSetMatchedNews(allNews,ruleSet):
    setRuleSetMatchedNews(allNews,ruleSet)
    filteredNews=[]
    for news in allNews:
        if ruleSet["name"] in news:
            filteredNews.append(news)
    return filteredNews

        
            
            
                
                    
            
            
                
            
        
    
    

In [7]:
# functions realted to data sets
def getRuleKeywords(rule):
    if("," in rule["conditions"]["keywords"] ):
        return [convertToLowerCase(x).strip() for x in rule["conditions"]["keywords"].split(",")]
    #keyword condition contains only one word
    return [rule["conditions"]["keywords"]]
def getRuleSetsKeywords(ruleSets):
    keywords=[]
    for ruleSet in ruleSets:
        for rule in ruleSet["rules"]:
            keywords=keywords+  getRuleKeywords(rule)
    return list(map(convertToLowerCase, keywords))

In [8]:
#functions related to report
headers=["id","url","name","lang","type","tags","categories","title","description","content","crawl_date","modified_date","published_date","text","rules"]
def createExcel(results,ruleSetName):
    
    # Call a Workbook() function of openpyxl  
    # to create a new blank Workbook object 
    wb = openpyxl.Workbook() 

    # Get workbook active sheet   
    # from the active attribute.  
    sheet = wb.active 

    # writing to the specified cell 


    for i in range(len(headers)):
        sheet.cell(row = 1, column = i+1).value = headers[i]  
    for k in range(len(results)):
        for j in range(len(headers)-1):
            sheet.cell(row = k+2, column = j+1).value = str(results[k].get(headers[j] ,"") ) 
        sheet.cell(row = k+2, column = len(headers)).value = str(results[k].get(ruleSetName ,"") )

    # set the height of the row 
    for l in range(len(results)):
        sheet.row_dimensions[l+1].height = 50
    # set the width of the column 
    width=30
    width3=50
    width2=70
    sheet.column_dimensions['A'].width = width
    sheet.column_dimensions['B'].width = width3
    sheet.column_dimensions['C'].width = width
    sheet.column_dimensions['G'].width = width
    sheet.column_dimensions['H'].width = width2
    sheet.column_dimensions['I'].width = width2
    sheet.column_dimensions['K'].width = width
    sheet.column_dimensions['L'].width = width
    sheet.column_dimensions['M'].width = width
    sheet.column_dimensions['N'].width = 100
    sheet.column_dimensions['O'].width = width2


    # save the file 
    print(ruleSetName+'.xlsx has created')
    wb.save(ruleSetName+'.xlsx') 
def createAllExcelFiles(allNews,ruleSets):
    for ruleSet in ruleSets:
        matchedNews=getRuleSetMatchedNews(allNews,ruleSet)
        createExcel(matchedNews,ruleSet["name"])

In [9]:
#main functions
def getFormatedNews():
    response = requests.get("http://mock.artiwise.com/api/news?_page=0&_limit=500")
    allNews=response.json()
    allNews=getNewsWithText(allNews)
    return allNews

In [10]:
# fetch data from servise and add normalized text prop
allNews=getFormatedNews()
# create keywords tree from rulsets
kwtree=getAhoCorasickTree(getRuleSetsKeywords(ruleSets))
# news that contains keywords
filteredNews=getFilteredNews(allNews,kwtree)
# create excel file with rule set name
createAllExcelFiles(filteredNews,ruleSets) 

Dünya Sağlık Örgütü Duyuruları.xlsx has created
coronavirus english news and drug.xlsx has created
